In [ ]:
!pip install pyomo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
!apt-get install -y coinor-cbc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 http:/

In [ ]:
from pygments import highlight
from pygments.lexers import ScalaLexer
from pygments.formatters import HtmlFormatter

# Define file paths
input_file_path = "/content/SinglePendulumNN.scala"
output_file_path = "/content/Extracted_SinglePendulumNN.html"

# Read the content of the Scala file
with open(input_file_path, "r") as input_file:
    content = input_file.read()

# Highlight the Scala code in HTML format
formatter = HtmlFormatter(full=True, linenos=True, style="colorful")
highlighted_code = highlight(content, ScalaLexer(), formatter)

# Write the highlighted code to an HTML file
with open(output_file_path, "w") as output_file:
    output_file.write(highlighted_code)

output_file_path  # Path to the generated HTML file


'/content/Extracted_SinglePendulumNN.html'

In [ ]:
!pip install beautifulsoup4


In [ ]:
import re
import numpy as np
from pyomo.environ import *
from bs4 import BeautifulSoup

# Load HTML file content
def load_html_file(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        html_content = file.read()
    return html_content

# Parse HTML file to extract weights
def parse_weights_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    # Example: assume weights are in <pre> tags or similar structure
    weight_matrices = soup.find_all('pre')  # Adjust based on the actual HTML structure
    parsed_weights = []

    for matrix in weight_matrices:
        rows = re.findall(r"List\((.*?)\)", matrix.get_text())

        if not rows:
            print("Warning: Found an empty or malformed matrix. Skipping...")
            continue

        try:
            parsed_matrix = [[float(value) for value in row.split(',')] for row in rows]
            parsed_weights.append(parsed_matrix)
        except ValueError:
            print("Warning: Failed to parse a matrix due to non-numeric values. Skipping...")

    # Ensure all matrices are of consistent shape
    validated_weights = []
    for matrix in parsed_weights:
        row_lengths = {len(row) for row in matrix}
        if len(row_lengths) == 1:  # All rows have the same length
            validated_weights.append(matrix)
        else:
            print("Warning: Inconsistent row lengths in matrix. Skipping this matrix.")

    return validated_weights

# Quantize weights with Pyomo MILP
def quantize_weights_with_pyomo(weights, error_bound=0.01, min_frac_bits=5, max_frac_bits=32):
    num_layers = len(weights)
    model = ConcreteModel()

    # Define binary variables for each possible bit width for each layer
    possible_bits = list(range(min_frac_bits, max_frac_bits + 1))
    model.frac_bits = Var(range(num_layers), within=Integers, bounds=(min_frac_bits, max_frac_bits))
    model.select_bits = Var(range(num_layers), possible_bits, within=Binary)
    model.quant_errors = Var(range(num_layers), within=NonNegativeReals)

    # Objective: Minimize the sum of fractional bits across all layers
    model.objective = Objective(expr=sum(model.frac_bits[i] for i in range(num_layers)), sense=minimize)

    # Constraints for selecting fractional bits
    for i in range(num_layers):
        # Ensure only one fractional bit choice is active per layer
        model.add_component(f"one_bit_choice_{i}", Constraint(expr=sum(model.select_bits[i, b] for b in possible_bits) == 1))

        # Link fractional bit choice to the fractional bits variable
        model.add_component(f"frac_bit_value_{i}", Constraint(
            expr=model.frac_bits[i] == sum(b * model.select_bits[i, b] for b in possible_bits)
        ))

        # Define quantization error as 1 / (2 ** frac_bits) for each layer
        model.add_component(f"quant_error_bound_{i}", Constraint(
            expr=model.quant_errors[i] <= error_bound
        ))
        model.add_component(f"error_constraint_{i}", Constraint(
            expr=model.quant_errors[i] <= sum((1 / (2 ** b)) * model.select_bits[i, b] for b in possible_bits)
        ))

    # Solve the model
    solver = SolverFactory('cbc')
    solver.solve(model)

    # Retrieve optimized fractional bits and quantized weights
    optimized_bits = [int(model.frac_bits[i].value) for i in range(num_layers)]
    quantized_weights = []

    for i, weight_matrix in enumerate(weights):
        frac_bits = optimized_bits[i]
        scale_factor = 2 ** frac_bits
        quantized_layer = np.round(np.array(weight_matrix) * scale_factor) / scale_factor
        quantized_weights.append(quantized_layer.tolist())

    return optimized_bits, quantized_weights

# Generate C++ code
def generate_cpp_code(optimized_bits, quantized_weights):
    cpp_code = "#include <iostream>\n#include <vector>\n\n"
    cpp_code += "class QuantizedNeuralNetwork {\npublic:\n"

    for i, weights in enumerate(quantized_weights):
        cpp_code += f"    // Layer {i+1} weights\n"
        cpp_code += f"    std::vector<std::vector<float>> weights_{i+1} = {{\n"

        for row in weights:
            cpp_code += "        { " + ", ".join(f"{value:.6f}f" for value in row) + " },\n"

        cpp_code += "    };\n\n"

    cpp_code += "    void forward(const std::vector<float>& input) {\n"
    cpp_code += "        // Add forward pass logic here\n"
    cpp_code += "    }\n};\n\n"

    cpp_code += "int main() {\n"
    cpp_code += "    QuantizedNeuralNetwork nn;\n"
    cpp_code += "    // Example input and forward call\n"
    cpp_code += "    std::vector<float> input = { /* input values */ };\n"
    cpp_code += "    nn.forward(input);\n"
    cpp_code += "    return 0;\n}\n"

    return cpp_code

# Main function to convert HTML NN to quantized C++ NN
def html_to_quantized_cpp(file_path):
    # Step 1: Load and parse HTML file
    html_content = load_html_file(file_path)
    parsed_weights = parse_weights_from_html(html_content)

    # Step 2: Quantize weights
    optimized_bits, quantized_weights = quantize_weights_with_pyomo(parsed_weights)

    # Step 3: Generate C++ code
    cpp_code = generate_cpp_code(optimized_bits, quantized_weights)
    return cpp_code

# File path to your HTML neural network file
file_path = '/content/Extracted_SinglePendulumNN.html'  # Replace with actual path
cpp_code = html_to_quantized_cpp(file_path)

# Save to a C++ file
with open('QuantizedNeuralNetwork.cpp', 'w') as cpp_file:
    cpp_file.write(cpp_code)

print("Quantized C++ code has been saved to QuantizedNeuralNetwork.cpp.")


Quantized C++ code has been saved to QuantizedNeuralNetwork.cpp.


Error: Quantized output is empty. Check the C++ executable output.
Quantized output is unavailable for comparison.
